# Python test

TBD

# SQL test

I opted for a SQLLite solution here to keep it simple and portable. See the workings below and then answers at the end...

In [5]:
import sqlite3
db = sqlite3.connect(':memory:')

cursor = db.cursor()
cursor.execute('''
CREATE TABLE prices(product TEXT, price_effective_date DATE, price INT)
''')
db.commit()

prices = [("product_1", "01/01/2018", 50),
("product_2", "01/01/2018", 40),
("product_1", "03/01/2018", 25),
("product_2", "05/01/2018", 20),
("product_1", "10/01/2018", 50),
("product_2", "12/01/2018", 40)]
cursor.executemany('''
INSERT INTO prices(product, price_effective_date, price) VALUES(?,?,?)
''', prices)
db.commit()

cursor = db.cursor()
cursor.execute('''
CREATE TABLE sales(product TEXT, sales_date DATE, quantity INT)
''')
db.commit()

sales = [("product_1", "01/01/2018", 10),
("product_2", "02/01/2018", 12),
("product_1", "04/01/2018", 50),
("product_2", "06/01/2018", 70),
("product_1", "12/01/2018", 8),
("product_2", "15/01/2018", 9)]

cursor.executemany('''
INSERT INTO sales(product, sales_date, quantity) VALUES(?,?,?)
''', sales)
db.commit()

total_sql = """
SELECT SUM(p.price * s.quantity)
FROM sales s
INNER JOIN (
SELECT product, price_effective_date, price
, COALESCE(MIN(next_price_effective_date),'99/99/9999') AS next_price_effective_date
FROM (
SELECT p1.product, p1.price_effective_date, p1.price
, p2.price_effective_date AS next_price_effective_date
FROM prices p1
LEFT OUTER JOIN prices p2
ON p1.product = p2.product
AND p1.price_effective_date < p2.price_effective_date
ORDER by p1.product, p1.price_effective_date
) x
GROUP BY product, price_effective_date, price
) p
ON s.product = p.product
AND s.sales_date >= p.price_effective_date
AND s.sales_date < p.next_price_effective_date
"""

cursor.execute(total_sql)
total_result = cursor.fetchone()

db.close()

## Question 1

In [9]:
print(total_result)

[(4390,)]


## Question 2

In [7]:
print(total_sql)


SELECT SUM(p.price * s.quantity)
FROM sales s
INNER JOIN (
SELECT product, price_effective_date, price
, COALESCE(MIN(next_price_effective_date),'99/99/9999') AS next_price_effective_date
FROM (
SELECT p1.product, p1.price_effective_date, p1.price
, p2.price_effective_date AS next_price_effective_date
FROM prices p1
LEFT OUTER JOIN prices p2
ON p1.product = p2.product
AND p1.price_effective_date < p2.price_effective_date
ORDER by p1.product, p1.price_effective_date
) x
GROUP BY product, price_effective_date, price
) p
ON s.product = p.product
AND s.sales_date >= p.price_effective_date
AND s.sales_date < p.next_price_effective_date

